In [1]:
%pip install torcheeg
%pip install moabb==0.5


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Load moabb dataset
import torcheeg.datasets.moabb as moabb_dataset
from torcheeg import transforms
from moabb.datasets import BNCI2014001
from moabb.paradigms import LeftRightImagery

moabb = BNCI2014001()
moabb.subject_list = [1, 2, 3]
paradigm = LeftRightImagery()

2024-01-24 20:50:04.537708: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-24 20:50:04.810371: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-24 20:50:04.874129: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-24 20:50:04.874143: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

Tensorflow not install, you could not use those pipelines
To use the get_shape_from_baseconcar, InputShapeSetterEEG, BraindecodeDatasetLoaderyou need to install `braindecode`.`pip install braindecode` or Please refer to `https://braindecode.org`.


In [3]:
dataset = moabb_dataset.MOABBDataset(
    dataset=moabb,
    paradigm=paradigm,
    io_path='./io/moabb',
    # offline_transform=transforms.Compose([transforms.BandDifferentialEntropy()]),
    online_transform=transforms.ToTensor(),
    label_transform=transforms.Compose([
        transforms.Select('label'),
        transforms.Mapping({'left_hand': 0, 'right_hand': 1,}),
    ])
)

[2024-01-24 20:50:06] INFO (torcheeg/MainThread) 🔍 | Detected cached processing results, reading cache from ./io/moabb.


In [4]:
dataset[2]

(tensor([[ 3.3079,  3.0014,  2.1278, -1.4036],
         [ 3.2340,  3.1799,  2.2855, -2.5170],
         [ 3.2573,  3.0450,  2.1432, -1.8323],
         [ 3.3858,  3.1501,  2.3015, -1.3530],
         [ 3.4854,  3.1256,  2.2190, -1.9134],
         [ 3.6255,  3.3306,  2.3606, -3.1138],
         [ 3.1585,  2.9723,  2.0644, -2.8291],
         [ 3.3503,  3.1082,  2.2270, -2.3801],
         [ 3.4040,  3.0672,  2.1147, -2.0747],
         [ 3.5860,  3.1837,  2.2757, -1.7749],
         [ 3.7133,  3.1518,  2.1847, -3.2434],
         [ 3.9287,  3.2827,  2.3741, -2.7579],
         [ 3.9143,  3.5193,  2.4629, -1.5646],
         [ 3.5198,  3.1574,  2.2032, -2.3894],
         [ 3.5965,  3.0906,  2.1086, -2.5781],
         [ 3.7583,  3.1620,  2.2260, -2.5988],
         [ 3.8750,  3.1585,  2.1983, -3.8501],
         [ 4.0504,  3.2456,  2.2810, -2.6974],
         [ 3.7864,  3.1971,  2.1674, -2.6001],
         [ 3.8891,  3.1942,  2.1937, -2.9871],
         [ 3.9816,  3.1584,  2.2129, -4.0444],
         [ 4.

In [5]:
import torch
import torch.nn as nn
from ncps.wirings import AutoNCP
from ncps.torch import CfC

class OnlyLiquidEEG(nn.Module):
    def __init__(self, liquid_units=50, num_classes=2, channels=4):
        super().__init__()
        self.liquid_block = LiquidBlock(units=liquid_units, out_features=num_classes, in_features=channels)
        self.softmax = nn.Softmax(dim=0)

    def forward(self, x, state=None):
        x, hx = self.liquid_block.forward(torch.squeeze(x, dim=1), state)
        return self.softmax(x), hx



class LiquidBlock(nn.Module):
    def __init__(self, units=20, out_features=10, in_features=5):
        super().__init__()
        wiring = AutoNCP(units, out_features)
        self.units = units
        self.liquid = CfC(in_features, wiring, return_sequences=False, batch_first=True)

    def forward(self, x, state=None):
        x, hx = self.liquid.forward(input=x, hx=state)
        return x, hx

In [6]:
import os
import time
import logging

os.makedirs('./examples_vanilla_torch/log', exist_ok=True)
logger = logging.getLogger('Training models with vanilla PyTorch')
logger.setLevel(logging.DEBUG)
console_handler = logging.StreamHandler()
timeticks = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
file_handler = logging.FileHandler(
    os.path.join('./examples_vanilla_torch/log', f'{timeticks}.log'))
logger.addHandler(console_handler)
logger.addHandler(file_handler)

In [7]:
import random
import numpy as np

import torch

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed_everything(42)

In [8]:
from torcheeg.model_selection import KFoldPerSubject

k_fold = KFoldPerSubject(n_splits=10,
                         split_path='./examples_vanilla_torch/split',
                         shuffle=True)

In [9]:
import datetime

device = "cuda" if torch.cuda.is_available() else "cpu"

# training process
def train(dataloader, model, loss_fn, optimizer):
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    total_start = datetime.datetime.now()
    size = len(dataloader.dataset)

    last_batch = 0
    start_batch_loading = datetime.datetime.now()
    for batch_idx, (X, y) in enumerate(dataloader):
        start_batch_training = datetime.datetime.now()
        batch_loading_time = (start_batch_training - start_batch_loading).total_seconds()
  
        X, y = X.to(device), y.to(device)

        # Compute prediction and loss
        pred, _ = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        batch_training_time = (datetime.datetime.now() - start_batch_training).total_seconds()
        start_batch_loading = datetime.datetime.now()
        if batch_idx % 100 == 0:
            loss, current = loss.item(), batch_idx * len(X)
            logger.info(f"Train loss: {loss:>7f}  [{current:>5d}/{size:>5d}] training time ratio: {batch_training_time / (batch_training_time + batch_loading_time)}")
    last_batch += 1
    print(f'Training time: {(datetime.datetime.now() - total_start).total_seconds()}')
    
    return loss

# # training process
# def train(dataloader, model, loss_fn, optimizer):
#     size = len(dataloader.dataset)
#     model.train()
#     for batch_idx, (X, y) in enumerate(dataloader):
#         X = X.to(device)
#         y = y.to(device)

#         # Compute prediction error
#         pred, _ = model(X)
#         loss = loss_fn(pred, y)

#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         if batch_idx % 100 == 0:
#             loss, current = loss.item(), batch_idx * len(X)
#             logger.info(f"Loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

#     return loss


def valid(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    loss, correct = 0, 0
    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)

            pred, _ = model(X)
            loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    loss /= num_batches
    correct /= size
    logger.info(f"Valid Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {loss:>8f}\n")

    return correct, loss

# validation process
# def valid(dataloader, model, loss_fn):
#     size = len(dataloader.dataset)
#     num_batches = len(dataloader)
#     model.eval()
#     loss, correct = 0, 0
#     with torch.no_grad():
#         for X, y in dataloader:
#             X = X.to(device)
#             y = y.to(device)

#             pred, _ = model(X)
#             loss += loss_fn(pred, y).item()
#             correct += (pred.argmax(1) == y).type(torch.float).sum().item()
#     loss /= num_batches
#     correct /= size
#     logger.info(
#         f"Valid Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {loss:>8f} \n"
#     )

#     return correct, loss

In [10]:
import torch.nn as nn

from torcheeg.model_selection import train_test_split
from torch.utils.data.dataloader import DataLoader

loss_fn = nn.CrossEntropyLoss()
batch_size = 64

test_accs = []
test_losses = []

for i, (train_dataset, test_dataset) in enumerate(k_fold.split(dataset)):
    # initialize model
    model = OnlyLiquidEEG(liquid_units=412).to(device)
    # initialize optimizer
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=1e-3)  # official: weight_decay=5e-1
    # split train and val
    train_dataset, val_dataset = train_test_split(
        train_dataset,
        test_size=0.2,
        split_path=f'./examples_vanilla_torch/split{i}',
        shuffle=True)
    train_loader = DataLoader(train_dataset,
                              batch_size=batch_size,
                              shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

    epochs = 50
    best_val_acc = 0.0
    for t in range(epochs):
        train_loss = train(train_loader, model, loss_fn, optimizer)
        val_acc, val_loss = valid(val_loader, model, loss_fn)
        # save the best model based on val_acc
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(),
                       f'./examples_vanilla_torch/model{i}.pt')

    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # load the best model to test on test set
    model.load_state_dict(torch.load(f'./examples_vanilla_torch/model{i}.pt'))
    test_acc, test_loss = valid(test_loader, model, loss_fn)

    # log the test result
    logger.info(
        f"Test Error {i}: \n Accuracy: {(100*test_acc):>0.1f}%, Avg loss: {test_loss:>8f}"
    )

    test_accs.append(test_acc)
    test_losses.append(test_loss)

# log the average test result on cross-validation datasets
logger.info(
    f"Test Error: \n Accuracy: {100*np.mean(test_accs):>0.1f}%, Avg loss: {np.mean(test_losses):>8f}"
)

[2024-01-24 20:50:07] INFO (torcheeg/MainThread) 📊 | Detected existing split of train and test set, use existing split from ./examples_vanilla_torch/split.
[2024-01-24 20:50:07] INFO (torcheeg/MainThread) 💡 | If the dataset is re-generated, you need to re-generate the split of the dataset instead of using the previous split.
[2024-01-24 20:50:08] INFO (torcheeg/MainThread) Read the split of train and test set from ./examples_vanilla_torch/split0. If you want to use the same setting for the experiment, please set split_path to ./examples_vanilla_torch/split0 for the next run.
Train loss: 0.693170  [    0/  207] training time ratio: 0.95667170171003
Valid Error: 
 Accuracy: 51.9%, Avg loss: 0.693095

Train loss: 0.692985  [    0/  207] training time ratio: 0.8576677393016665


Training time: 0.376825


Valid Error: 
 Accuracy: 57.7%, Avg loss: 0.693066

Train loss: 0.692736  [    0/  207] training time ratio: 0.8571428571428572
Valid Error: 
 Accuracy: 51.9%, Avg loss: 0.693120



Training time: 0.160522
Training time: 0.157777


Train loss: 0.692899  [    0/  207] training time ratio: 0.8324868185556039
Valid Error: 
 Accuracy: 51.9%, Avg loss: 0.693023

Train loss: 0.692880  [    0/  207] training time ratio: 0.8495970210161192
Valid Error: 
 Accuracy: 69.2%, Avg loss: 0.692562



...

Train loss: 0.693145  [    0/  208] training time ratio: 0.8602859732151849
Valid Error: 
 Accuracy: 42.3%, Avg loss: 0.693148

Train loss: 0.693145  [    0/  208] training time ratio: 0.852600469694171
Valid Error: 
 Accuracy: 42.3%, Avg loss: 0.693148



Training time: 0.159266
Training time: 0.148774


Train loss: 0.693145  [    0/  208] training time ratio: 0.8570779748529904
Valid Error: 
 Accuracy: 42.3%, Avg loss: 0.693147

Train loss: 0.693146  [    0/  208] training time ratio: 0.8518538191862318
Valid Error: 
 Accuracy: 42.3%, Avg loss: 0.693148

Valid Error: 
 Accuracy: 46.4%, Avg loss: 0.693157

Test Error 29: 
 Accuracy: 46.4%, Avg loss: 0.693157


Training time: 0.160489
Training time: 0.148708


Test Error: 
 Accuracy: 53.0%, Avg loss: 0.692324
